# Import and Dataset load

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense , Flatten ,Embedding,Input,LSTM, Bidirectional, Dense, Dropout,GRU,SpatialDropout1D, Conv1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.initializers import Constant
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file = '/content/drive/MyDrive/Paper Writing/cleaned_data.csv'
data = pd.read_csv(file, encoding='utf-8')

In [ ]:
data.head()

,Unnamed: 0,sentence,hate,category,cleaned_text,sentenceLength,ReviewChars
0,0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী,4,30
1,1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার,6,33
2,2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,20,107
3,3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,7,35
4,4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ তুর মার হেডায় খেলবে সাকিব,10,48


# Machine Learning Classifiers

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['hate'],random_state = 0,test_size = 0.20)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((24000,), (6000,), (24000,), (6000,))

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=4000)
Tfidf_vect.fit(data['cleaned_text'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [ ]:
model = MultinomialNB()
model.fit(Train_X_Tfidf,y_train)
from sklearn.metrics import accuracy_score

predictions_train_NB = model.predict(Train_X_Tfidf)
predictions_NB = model.predict(Test_X_Tfidf)
print("Naive Bayes Training Accuracy Score -> ",accuracy_score(predictions_train_NB, y_train)*100)
print("Naive Bayes Test Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)

Naive Bayes Training Accuracy Score ->  74.75
Naive Bayes Test Accuracy Score ->  73.33333333333333


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test,predictions_NB))
print(confusion_matrix(y_test,predictions_NB))

              precision    recall  f1-score   support

           0       0.73      0.96      0.83      4021
           1       0.76      0.28      0.41      1979

    accuracy                           0.73      6000
   macro avg       0.74      0.62      0.62      6000
weighted avg       0.74      0.73      0.69      6000

[[3844  177]
 [1423  556]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF=RandomForestClassifier(n_estimators=100)
RF.fit(Train_X_Tfidf,y_train)
pred_y_train = RF.predict(Train_X_Tfidf)
pred_y = RF.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Train Accuracy Score -> ",accuracy_score(pred_y_train, y_train)*100)
print("Test Accuracy Score -> ",accuracy_score(pred_y, y_test)*100)

Train Accuracy Score ->  94.55
Test Accuracy Score ->  73.88333333333334


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,pred_y))
cf_matrix = confusion_matrix(y_test, pred_y)
print(cf_matrix)

              precision    recall  f1-score   support

           0       0.75      0.92      0.83      4021
           1       0.70      0.37      0.48      1979

    accuracy                           0.74      6000
   macro avg       0.72      0.65      0.65      6000
weighted avg       0.73      0.74      0.71      6000

[[3702  319]
 [1248  731]]


In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

clf = clf.fit(Train_X_Tfidf,y_train)

pred_y_train = clf.predict(Train_X_Tfidf)
pred_y = clf.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Train Accuracy Score -> ",accuracy_score(pred_y_train, y_train)*100)
print("Test Accuracy Score -> ",accuracy_score(pred_y, y_test)*100)

Train Accuracy Score ->  94.55
Test Accuracy Score ->  68.7


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,pred_y))
cf_matrix = confusion_matrix(y_test, pred_y)
print(cf_matrix)

              precision    recall  f1-score   support

           0       0.75      0.80      0.77      4021
           1       0.53      0.45      0.49      1979

    accuracy                           0.69      6000
   macro avg       0.64      0.63      0.63      6000
weighted avg       0.68      0.69      0.68      6000

[[3230  791]
 [1087  892]]


In [ ]:
from sklearn import svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,y_train)
pred_y_train = SVM.predict(Train_X_Tfidf)
pred_y = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Train Accuracy Score -> ",accuracy_score(pred_y_train, y_train)*100)
print("SVM Test Accuracy Score -> ",accuracy_score(pred_y, y_test)*100)

SVM Train Accuracy Score ->  76.27916666666667
SVM Test Accuracy Score ->  75.38333333333334


# Word Embedding

In [ ]:
corpus=[]
words = 0;
j = 0
for i in data['cleaned_text'].values:
    corpus.append(str(i).split(" "))
    words += len(corpus[j])
    j += 1
print("Total words = ", words)
print("Total Sentances = ", len(corpus))

Total words =  509593
Total Sentances =  30000


In [ ]:
from bnlp import BengaliWord2Vec
import gensim
from gensim.models import Word2Vec

bwv = BengaliWord2Vec()

trained_model_path = "/content/drive/MyDrive/Paper Writing/models/bangla_word2vec/bnwiki_word2vec.model"
data_file = corpus
model_name = "/content/drive/MyDrive/Paper Writing/models/new_w2v.model"
vector_name = "/content/drive/MyDrive/Paper Writing/models/new_w2v_vector.vector"
bwv.pretrain(trained_model_path, data_file, model_name, vector_name, epochs=10)


w2v_model = Word2Vec.load("/content/drive/MyDrive/Paper Writing/models/new_w2v.model")
w2v_model.train(corpus,total_words=509593, epochs=10)

w2v_model.wv.most_similar('নষ্ট')
w2v_model.wv['বাংলাদেশ'].shape

ℹ model loading ....
ℹ vocab building with new sentences
ℹ pre-training started.......
ℹ please wait.....it will take time according to your data size and
computation capability
✔ pre-train completed successfully
✔ pre-trianing loss: 0.0
ℹ model and vector saving...
✔ model and vector saved as /content/drive/MyDrive/Paper
Writing/models/new_w2v.model and /content/drive/MyDrive/Paper
Writing/models/new_w2v_vector.vector


(100,)

In [ ]:
w2v_model.wv.most_similar('নষ্ট')

[('বিনষ্ট', 0.7570128440856934),
 ('নিঃশেষ', 0.6772664785385132),
 ('ধ্বংস', 0.6578051447868347),
 ('ধংস', 0.6071178317070007),
 ('বিকৃত', 0.6030839085578918),
 ('লাঘব', 0.5986009836196899),
 ('ছিন্নভিন্ন', 0.596372127532959),
 ('পূরণ', 0.5908770561218262),
 ('নস্ট', 0.5903378129005432),
 ('ধ্বংশ', 0.5859116911888123)]

In [ ]:
filename = '/content/drive/MyDrive/Paper Writing/models/ban_hate_w2v.txt'
w2v_model.wv.save_word2vec_format(filename, binary=False)

In [ ]:
from sklearn.model_selection import train_test_split
import os

# tokenize every words so that evey words maps to numaric value
tok = Tokenizer()
tok.fit_on_texts(data['cleaned_text'].values.astype('U'))
encd_rev = tok.texts_to_sequences(data['cleaned_text'].values.astype('U'))
# tok.word_index

# make all the input sentance same length with add padding
max_rev_len = 60 # max lenght of a sentance
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim = 100 # embedding dimension 
pad_rev= pad_sequences(encd_rev, maxlen=60, padding='post')

print("Size of vocabulary : ", vocab_size)
print("All sentances with same length ", pad_rev.shape)

# lebel encode the output label to categorical
Y = data['hate']
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)

# print(encoder.classes_)
# print(dummy_y)

# test train split
X_train,X_test,y_train,y_test=train_test_split(pad_rev,encoded_Y,test_size=0.2, random_state = 0)


# make a dictionary. word as key and feature vector as value
embedding_index={}
f = open('/content/drive/MyDrive/Paper Writing/models/ban_hate_w2v.txt',encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:])
    embedding_index[word]=coefs
f.close()

# create a embeddings matrix with 200 dimenstion
EMBEDDING_DIM=100
embedding_matrix=np.zeros((vocab_size,EMBEDDING_DIM))
for word, i in tok.word_index.items():
    if i>vocab_size:
        continue
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

Size of vocabulary :  47752
All sentances with same length  (30000, 60)


In [ ]:
print(X_train)
print(y_train)

[[  237    17   795 ...     0     0     0]
 [  507 29041   134 ...     0     0     0]
 [  224   783    22 ...     0     0     0]
 ...
 [   99   364    40 ...     0     0     0]
 [   67 11539   250 ...     0     0     0]
 [21263 21264   351 ...     0     0     0]]
[1 0 1 ... 1 0 1]


In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((24000, 60), (6000, 60), (24000,), (6000,))

In [ ]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
print(y_train.shape, y_test.shape)

(24000, 1) (6000, 1)


In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X_train, y_train,test_size=0.2, random_state = 0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((19200, 60), (4800, 60), (19200, 1), (4800, 1))

# Deep learning Model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
def print_result(model, history, x_test, y_test, name):
    model_path = "/content/drive/MyDrive/Paper Writing/models/"+ name+ " model"
    
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 

    # save to json:  
    hist_json_file = "/content/drive/MyDrive/Paper Writing/History/" + name + 'history.json' 
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)
            
    #save model
    model.save(model_path)
    
    
    #ploting training  history
    accr = model.evaluate(x_test,y_test)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

    y_pred = model.predict(x_test)

    #normalize
    y_pred_frac=np.argmax(y_pred, axis=1)
    y_test_frac=np.argmax(y_test, axis=1)

    print("-------------Classification Report----------------")
    print(classification_report(y_test_frac, y_pred_frac))
    print("-------------------//*//-------------------------")

    print("-------------Confusion Matrix----------------")
    cm = confusion_matrix(y_test_frac, y_pred_frac)
    print(cm)
    print("-------------------//*//-------------------------")
  

In [ ]:
from keras.layers import Layer
from keras import backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
# , X_train_word, y_train_word, x_test, y_test, x_val, y_val
def train_model(model):
    epochs = 10
    batch_size = 64
    #callbacks = [EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_data=(X_val, y_val))
    # history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    return history, model

In [ ]:
model=Sequential()
embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=60,
                        trainable=False)
model.add(embedding_layer)
model.add(Conv1D(128, 5, activation='relu'))
model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25,recurrent_dropout=0.25)))
model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25,recurrent_dropout=0.25)))
model.add(attention())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print('summary of the built model..')
model.summary()

summary of the built model..
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 60, 100)           4775200   
                                                                 
 bidirectional_2 (Bidirectio  (None, 60, 256)          176640    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 60, 256)          296448    
 nal)                                                            
                                                                 
 attention_1 (attention)     (None, 256)               316       
                                                                 
 dense_1 (Dense)             (None, 200)               51400     
                                                                 
 dropout (Dropout)       

In [ ]:
# history, model = train_model(model, X_train_word, y_train_word, X_test_word, y_test_word, x_val, y_val)
history, model = train_model(model)

Epoch 1/10
300/300 [==============================] - 326s 1s/step - loss: 0.4131 - accuracy: 0.8145 - val_loss: 0.3839 - val_accuracy: 0.8302
Epoch 2/10
300/300 [==============================] - 312s 1s/step - loss: 0.3548 - accuracy: 0.8480 - val_loss: 0.3591 - val_accuracy: 0.8444
Epoch 3/10
300/300 [==============================] - 312s 1s/step - loss: 0.3316 - accuracy: 0.8621 - val_loss: 0.3446 - val_accuracy: 0.8573
Epoch 4/10
300/300 [==============================] - 312s 1s/step - loss: 0.3089 - accuracy: 0.8726 - val_loss: 0.3386 - val_accuracy: 0.8556
Epoch 5/10
300/300 [==============================] - 309s 1s/step - loss: 0.2932 - accuracy: 0.8755 - val_loss: 0.3438 - val_accuracy: 0.8565
Epoch 6/10
300/300 [==============================] - 309s 1s/step - loss: 0.2779 - accuracy: 0.8829 - val_loss: 0.3565 - val_accuracy: 0.8600
Epoch 7/10
300/300 [==============================] - 308s 1s/step - loss: 0.2619 - accuracy: 0.8913 - val_loss: 0.3493 - val_accuracy: 0.8606

In [ ]:
print_result(model, history, X_test, y_test, 'Bi-GRU-Bi-GRU_att_dense')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Paper Writing/models/Bi-GRU-Bi-GRU_att_dense model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Paper Writing/models/Bi-GRU-Bi-GRU_att_dense model/assets


188/188 [==============================] - 17s 83ms/step - loss: 0.4089 - accuracy: 0.8567
Test set
  Loss: 0.409
  Accuracy: 0.857
-------------Classification Report----------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6000

    accuracy                           1.00      6000
   macro avg       1.00      1.00      1.00      6000
weighted avg       1.00      1.00      1.00      6000

-------------------//*//-------------------------
-------------Confusion Matrix----------------
[[6000]]
-------------------//*//-------------------------


In [ ]:
from keras.layers import Layer
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
x = SpatialDropout1D(0.2)(embedding_sequences)
x = LSTM(64, dropout=0.2,  return_sequences=True, recurrent_dropout=0.2)(x)
x = attention()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(sequence_input, outputs)